# Generate tiles for high resolution global extent map

[Data sources](https://docs.google.com/document/d/1D9kGrF9AUXDrMLqm-51ZfI43vlfYqpzcQd3mk7MIQqk/edit#heading=h.z8hx51w9ls2m)


In [1]:
import os
from pathlib import Path
import geopandas
import subprocess
import logging
import requests
import gzip
import fiona
from typing import Union

/opt/conda/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets'
logging.basicConfig(level=logging.INFO)

# @TODO: Add expected data files source as an environment variable. and add the download bit for the data sources.
assert BASE_DIR == '/home/jovyan/work/datasets', f'{BASE_DIR} is not the correct directory'
outFolder= Path(f'{BASE_DIR}/hi_res_extent')
#outputPath = f'{outFolder.parents[1]}/processed'

In [3]:
!ls {outFolder}

gmw_v4_sen2_sgl_cls_v007.gpkg		gmw_v4_sen2_sgl_cls_v007_wgs84.mbtiles
gmw_v4_sen2_sgl_cls_v007_wgs84.geojson	gmw_v4_sen2_sgl_cls_v007_ZG.mbtiles


In [8]:
df_extent = geopandas.read_file(f'{outFolder}/gmw_v4_sen2_sgl_cls_v007.gpkg')
df_extent.head(2)

,ClassID,geometry
0,1,"POLYGON ((35.66950 22.94630, 35.66960 22.94630..."
1,1,"POLYGON ((35.66620 22.94670, 35.66620 22.94660..."


In [9]:
df_extent.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
#df_extent.to_crs(epsg=4326, inplace=True)

In [10]:
df_extent.to_file(f'{outFolder}/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson', driver='GeoJSON')

In [4]:
data_path = f'{outFolder}/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson'
output_path = f'{outFolder}/gmw_v4_sen2_sgl_cls_v007_ZG.mbtiles'

!tippecanoe -zg -f -P -o {output_path} --extend-zooms-if-still-dropping {data_path}
print("Done!")

For layer 0, using name "gmw_v4_sen2_sgl_cls_v007_wgs84"
/home/jovyan/work/datasets/hi_res_extent/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson:801196: Found ] at top level
/home/jovyan/work/datasets/hi_res_extent/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson:899334: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
3342900 features, 603220067 bytes of geometry, 10028704 bytes of separate metadata, 48 bytes of string pool
Choosing a maxzoom of -z10 for features about 414 feet (126 meters) apart
Choosing a maxzoom of -z12 for resolution of about 72 feet (21 meters) within features
  99.9%  12/2817/1768  


Done!


In [9]:
username = 'globalmangrovewatch'
token = 'pk.eyJ1IjoiZ2xvYmFsbWFuZ3JvdmV3YXRjaCIsImEiOiJjbGIybnZsMW4wNjNqM3BucGJ6NXVzZHBiIn0.LTOZXrFVilMBleaeHv9NgQ'
cred_url = f'https://api.mapbox.com/uploads/v1/{username}/credentials'
cred = requests.post(cred_url, params={'access_token': token}).json()
cred

{'message': 'Not Found'}

In [3]:
def mbtilesGeneration(data_path: Path, output_path: Union[Path, None] = None,

 update: bool = False) -> Path:
    """
    Simplify geometry of a GeoDataFrame using tippecanoe.
    Parameters
    ----------
    data_path : Path - The path to the GeoDataFrame to simplify.
    output_path : Path - The path to the output GeoDataFrame.
    update : bool, optional - If True, the output GeoDataFrame will be overwritten.
                            The default is False.
    
    Returns
    -------
    Path - The path to the generated mbtiles file.
    """
    try:
        assert data_path.exists(), 'Data path does not exist.'
        if not output_path:
            output_path = data_path.with_suffix('.mbtiles')
        
        if update or not output_path.exists():
            
            if data_path.suffix != '.json':
                CMD = f'mapshaper {data_path} -clean allow-overlaps rewind -o format=geojson {data_path.with_suffix(".json")} force'
                subprocess.run(CMD ,shell=True, check=True)
                data_path = data_path.with_suffix('.json')

            assert data_path.suffix == '.json', 'Data path must be a json file.'
            
            logging.info('Creating mbtiles file...')
            
            subprocess.run(
                f"tippecanoe -zg -f -P -o {output_path} --extend-zooms-if-still-dropping {data_path}",
                shell=True, check=True
                )
        
        return output_path
    
    except Exception as e:
        logging.error(e)
        return 1

In [4]:
outFolder.exists()

True

In [4]:
for infile in outFolder.glob('bh_mangroves.gpkg'):
    logging.info(f'Processing {infile}')
    df = geopandas.read_file(infile)
    df = df.to_crs('EPSG:4326')
    df.to_file(f'{outFolder}/{infile.stem}.geojson', driver='GeoJSON')
    mbtilesGeneration(Path(f'{outFolder}/{infile.stem}.geojson'), Path(f'{outFolder}/{infile.stem}.mbtiles'))
    os.remove(f'{outFolder}/{infile.stem}.geojson')    


INFO:root:Processing /home/jovyan/work/datasets/National_Layers/bh_mangroves.gpkg
[clean] Retained 1 of 1 features


: 

: 

In [ ]:
for infile in outFolder.glob('*.shp'):
    logging.info(f'Processing {infile}')
    mbtilesGeneration(infile, update=True)

In [6]:
df_aus = geopandas.read_file(f'{outFolder}/AUS_mangrove_cover_2022.geojson')
df_aus.head()

,DN,geometry
0,3,"POLYGON ((142.05434 -10.35295, 142.05434 -10.3..."
1,3,"POLYGON ((142.05463 -10.35353, 142.05463 -10.3..."
2,2,"POLYGON ((142.05463 -10.35411, 142.05463 -10.3..."
3,3,"POLYGON ((142.05521 -10.35526, 142.05521 -10.3..."
4,3,"POLYGON ((142.05984 -10.35555, 142.05984 -10.3..."


In [8]:
df_aus = df_aus[df_aus['DN'] > 0]
df_aus.to_file(f'{outFolder}/AUS_mangrove_cover_2022.geojson', driver='GeoJSON')

In [9]:
for infile in outFolder.glob('*.geojson'):
    logging.info(f'Processing {infile}')
    mbtilesGeneration(infile, update=True)

INFO:root:Processing /home/jovyan/work/datasets/National_Layers/AUS_mangrove_cover_2022.geojson
[clean] Retained 986,345 of 986,345 features
[o] Wrote /home/jovyan/work/datasets/National_Layers/AUS_mangrove_cover_2022.json
INFO:root:Creating mbtiles file...
For layer 0, using name "AUS_mangrove_cover_2022"
/home/jovyan/work/datasets/National_Layers/AUS_mangrove_cover_2022.json:231926: Found ] at top level
/home/jovyan/work/datasets/National_Layers/AUS_mangrove_cover_2022.json:246033: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
986345 features, 55263998 bytes of geometry, 2959039 bytes of separate metadata, 52 bytes of string pool
Choosing a maxzoom of -z10 for features about 381 feet (117 meters) apart
Choosing a maxzoom of -z11 for resolution of about 192 feet (58 meters) within features
  99.9%  11/1896/1188  
INFO:root:Processing /home/jovyan/work/datasets/National_Layers/bh_mangroves.geojson


KeyboardInterrupt: 

In [6]:
!ls {outFolder} | grep mbtiles

AUS_mangrove_cover_2022.mbtiles
bh_mangroves.mbtiles
bh_mangroves.mbtiles-journal
car_mar_mangroves_2023.mbtiles
MangroveExtent2020-Kenya-Final-QA-v2.mbtiles
MangroveExtent2020-Madagascar-Final-QA-v3.mbtiles
MangroveExtent2020-Mozambique-Final-QA-v2.mbtiles
MangroveExtent2020-Tanzania-Final-QA-v2.mbtiles


In [10]:
df_aus = geopandas.read_file(f'{outFolder}/AUS_mangrove_cover_2022.geojson')
df_aus['DN'].value_counts()

2    498337
1    348510
3    139498
Name: DN, dtype: int64

In [ ]:
#!npm install -g @mapbox/mbview  

#!mbview $outputPath_mbtiles